In [1]:
!wget https://hadoop-and-big-data.s3-us-west-2.amazonaws.com/fitness-tracker/health_tracker_data_2020_1.json

--2020-04-17 16:20:20--  https://hadoop-and-big-data.s3-us-west-2.amazonaws.com/fitness-tracker/health_tracker_data_2020_1.json
Resolving hadoop-and-big-data.s3-us-west-2.amazonaws.com (hadoop-and-big-data.s3-us-west-2.amazonaws.com)... 52.218.197.81
Connecting to hadoop-and-big-data.s3-us-west-2.amazonaws.com (hadoop-and-big-data.s3-us-west-2.amazonaws.com)|52.218.197.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 305999 (299K) [application/json]
Saving to: ‘health_tracker_data_2020_1.json’

health_tracker_data 100%[===================>] 298.83K  1.60MB/s    in 0.2s    

2020-04-17 16:20:21 (1.60 MB/s) - ‘health_tracker_data_2020_1.json’ saved [305999/305999]



In [2]:
!wget https://hadoop-and-big-data.s3-us-west-2.amazonaws.com/fitness-tracker/health_tracker_data_2020_2.json

--2020-04-17 16:20:25--  https://hadoop-and-big-data.s3-us-west-2.amazonaws.com/fitness-tracker/health_tracker_data_2020_2.json
Resolving hadoop-and-big-data.s3-us-west-2.amazonaws.com (hadoop-and-big-data.s3-us-west-2.amazonaws.com)... 52.218.197.81
Connecting to hadoop-and-big-data.s3-us-west-2.amazonaws.com (hadoop-and-big-data.s3-us-west-2.amazonaws.com)|52.218.197.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 280434 (274K) [application/json]
Saving to: ‘health_tracker_data_2020_2.json’

health_tracker_data 100%[===================>] 273.86K  1.56MB/s    in 0.2s    

2020-04-17 16:20:25 (1.56 MB/s) - ‘health_tracker_data_2020_2.json’ saved [280434/280434]



In [3]:
!wget https://hadoop-and-big-data.s3-us-west-2.amazonaws.com/fitness-tracker/health_tracker_data_2020_3.json

--2020-04-17 16:20:36--  https://hadoop-and-big-data.s3-us-west-2.amazonaws.com/fitness-tracker/health_tracker_data_2020_3.json
Resolving hadoop-and-big-data.s3-us-west-2.amazonaws.com (hadoop-and-big-data.s3-us-west-2.amazonaws.com)... 52.218.220.89
Connecting to hadoop-and-big-data.s3-us-west-2.amazonaws.com (hadoop-and-big-data.s3-us-west-2.amazonaws.com)|52.218.220.89|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 306065 (299K) [application/json]
Saving to: ‘health_tracker_data_2020_3.json’

health_tracker_data 100%[===================>] 298.89K  1.07MB/s    in 0.3s    

2020-04-17 16:20:36 (1.07 MB/s) - ‘health_tracker_data_2020_3.json’ saved [306065/306065]



In [4]:
!ls

health_tracker_data_2020_1.json  Untitled1.ipynb
health_tracker_data_2020_2.json  Untitled.ipynb
health_tracker_data_2020_3.json


In [5]:
!rm -rf ../data/health_tracker

In [6]:
!mkdir -p ../data/health_tracker/raw

In [7]:
!mv health_tracker_data* ../data/health_tracker/raw

In [8]:
health_tracker = "../data/health_tracker/"

In [9]:
from os import environ

environ['PYSPARK_SUBMIT_ARGS'] = '--packages "io.delta:delta-core_2.11:0.5.0" pyspark-shell'

from pyspark import sql
spark = sql.SparkSession.builder \
        .master("local[8]") \
        .getOrCreate()
def display(dataframe):
    return dataframe.show()

In [10]:
raw_health_tracker_data_df = spark.read \
    .format("text") \
    .load(health_tracker + "raw/health_tracker_data_2020_1.json")

In [11]:
display(raw_health_tracker_data_df)

+--------------------+
|               value|
+--------------------+
|{"device_id":0,"h...|
|{"device_id":0,"h...|
|{"device_id":0,"h...|
|{"device_id":0,"h...|
|{"device_id":0,"h...|
|{"device_id":0,"h...|
|{"device_id":0,"h...|
|{"device_id":0,"h...|
|{"device_id":0,"h...|
|{"device_id":0,"h...|
|{"device_id":0,"h...|
|{"device_id":0,"h...|
|{"device_id":0,"h...|
|{"device_id":0,"h...|
|{"device_id":0,"h...|
|{"device_id":0,"h...|
|{"device_id":0,"h...|
|{"device_id":0,"h...|
|{"device_id":0,"h...|
|{"device_id":0,"h...|
+--------------------+
only showing top 20 rows



In [12]:
raw_health_tracker_data_df.printSchema()

root
 |-- value: string (nullable = true)



In [13]:
raw_health_tracker_data_df.write \
    .mode("append") \
    .format("delta") \
    .save(health_tracker + "bronze")

In [16]:
!rm ../data/health_tracker/raw/health_tracker_data_2020_1.json

In [17]:
bronze_health_tracker_data_df = spark.read \
    .format("delta") \
    .load(health_tracker + "bronze")

In [18]:
from pyspark.sql.functions import col, from_json

json_schema = "device_id int, heartrate double, name string, time float"
bronze_health_tracker_data_df = bronze_health_tracker_data_df \
    .withColumn("json", from_json(col("value"), json_schema))

In [19]:
bronze_health_tracker_data_df.printSchema()

root
 |-- value: string (nullable = true)
 |-- json: struct (nullable = true)
 |    |-- device_id: integer (nullable = true)
 |    |-- heartrate: double (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- time: float (nullable = true)



In [20]:
display(bronze_health_tracker_data_df)

+--------------------+--------------------+
|               value|                json|
+--------------------+--------------------+
|{"device_id":0,"h...|[0, 52.8139067501...|
|{"device_id":0,"h...|[0, 53.9078900098...|
|{"device_id":0,"h...|[0, 52.7129593616...|
|{"device_id":0,"h...|[0, 52.2880422685...|
|{"device_id":0,"h...|[0, 52.5156095386...|
|{"device_id":0,"h...|[0, 53.6280743846...|
|{"device_id":0,"h...|[0, 52.1760037066...|
|{"device_id":0,"h...|[0, 90.0456721836...|
|{"device_id":0,"h...|[0, 89.4695644522...|
|{"device_id":0,"h...|[0, 88.1490304138...|
|{"device_id":0,"h...|[0, 86.3092976213...|
|{"device_id":0,"h...|[0, 86.6672980008...|
|{"device_id":0,"h...|[0, 89.2044154758...|
|{"device_id":0,"h...|[0, 88.8691678316...|
|{"device_id":0,"h...|[0, 87.7252170634...|
|{"device_id":0,"h...|[0, 87.8357187561...|
|{"device_id":0,"h...|[0, 87.1769562648...|
|{"device_id":0,"h...|[0, 88.3403107931...|
|{"device_id":0,"h...|[0, 88.9083484735...|
|{"device_id":0,"h...|[0, 87.921

In [21]:
silver_health_tracker_data_df = bronze_health_tracker_data_df \
    .select(
        col("json.device_id"),
        col("json.heartrate"),
        col("json.name"),
        col("json.time"),
    )

In [23]:
display(silver_health_tracker_data_df)

+---------+-------------+--------------+------------+
|device_id|    heartrate|          name|        time|
+---------+-------------+--------------+------------+
|        0|52.8139067501|Deborah Powell| 1.5778368E9|
|        0|53.9078900098|Deborah Powell|1.57784038E9|
|        0|52.7129593616|Deborah Powell|1.57784397E9|
|        0|52.2880422685|Deborah Powell|1.57784755E9|
|        0|52.5156095386|Deborah Powell|1.57785114E9|
|        0|53.6280743846|Deborah Powell|1.57785485E9|
|        0|52.1760037066|Deborah Powell|1.57785843E9|
|        0|90.0456721836|Deborah Powell|1.57786202E9|
|        0|89.4695644522|Deborah Powell| 1.5778656E9|
|        0|88.1490304138|Deborah Powell|1.57786918E9|
|        0|86.3092976213|Deborah Powell|1.57787277E9|
|        0|86.6672980008|Deborah Powell|1.57787635E9|
|        0|89.2044154758|Deborah Powell|1.57788006E9|
|        0|88.8691678316|Deborah Powell|1.57788365E9|
|        0|87.7252170634|Deborah Powell|1.57788723E9|
|        0|87.8357187561|Deb

In [24]:
from pyspark.sql.functions import col, from_unixtime
 
silverDF = silver_health_tracker_data_df.withColumn("time", from_unixtime("time"))
silverDF = silverDF.withColumn("time", col("time").cast("timestamp"))
silverDF = silverDF.withColumn("dte", col("time").cast("date"))
silverDF = silverDF.withColumn("device_id", col("device_id").cast("integer"))


In [25]:
display(silver_health_tracker_data_df)

+---------+-------------+--------------+------------+
|device_id|    heartrate|          name|        time|
+---------+-------------+--------------+------------+
|        0|52.8139067501|Deborah Powell| 1.5778368E9|
|        0|53.9078900098|Deborah Powell|1.57784038E9|
|        0|52.7129593616|Deborah Powell|1.57784397E9|
|        0|52.2880422685|Deborah Powell|1.57784755E9|
|        0|52.5156095386|Deborah Powell|1.57785114E9|
|        0|53.6280743846|Deborah Powell|1.57785485E9|
|        0|52.1760037066|Deborah Powell|1.57785843E9|
|        0|90.0456721836|Deborah Powell|1.57786202E9|
|        0|89.4695644522|Deborah Powell| 1.5778656E9|
|        0|88.1490304138|Deborah Powell|1.57786918E9|
|        0|86.3092976213|Deborah Powell|1.57787277E9|
|        0|86.6672980008|Deborah Powell|1.57787635E9|
|        0|89.2044154758|Deborah Powell|1.57788006E9|
|        0|88.8691678316|Deborah Powell|1.57788365E9|
|        0|87.7252170634|Deborah Powell|1.57788723E9|
|        0|87.8357187561|Deb

In [26]:
silver_health_tracker_data_df.write \
   .mode("overwrite") \
   .format("delta") \
   .partitionBy("device_id") \
   .save(health_tracker + "silver")


In [29]:
!ls ../data/health_tracker/silver

 _delta_log    'device_id=1'  'device_id=3'
'device_id=0'  'device_id=2'  'device_id=4'


In [30]:
raw_health_tracker_data_df = spark.read \
    .format("text") \
    .load(health_tracker + "raw/health_tracker_data_2020_2.json")


In [31]:
raw_health_tracker_data_df.write \
 .mode("append") \
 .format("delta") \
 .save(health_tracker + "bronze")


In [32]:
!rm ../data/health_tracker/raw/health_tracker_data_2020_2.json

In [33]:
bronze_health_tracker_data_df = spark.read \
    .format("delta") \
    .load(health_tracker + "bronze")

In [34]:
from pyspark.sql.functions import col, from_json

json_schema = "device_id int, heartrate double, name string, time float"
bronze_health_tracker_data_df = bronze_health_tracker_data_df \
    .withColumn("json", from_json(col("value"), json_schema))

In [35]:
silver_health_tracker_data_df = bronze_health_tracker_data_df \
    .select(
        col("json.device_id"),
        col("json.heartrate"),
        col("json.name"),
        col("json.time"),
    )

In [36]:
from pyspark.sql.functions import col, from_unixtime
 
silverDF = silver_health_tracker_data_df.withColumn("time", from_unixtime("time"))
silverDF = silverDF.withColumn("time", col("time").cast("timestamp"))
silverDF = silverDF.withColumn("dte", col("time").cast("date"))
silverDF = silverDF.withColumn("device_id", col("device_id").cast("integer"))


In [37]:
silver_health_tracker_data_df.write \
   .mode("append") \
   .format("delta") \
   .partitionBy("device_id") \
   .save(health_tracker + "silver")


In [38]:
!ls ../data/health_tracker/silver

 _delta_log    'device_id=1'  'device_id=3'
'device_id=0'  'device_id=2'  'device_id=4'


In [39]:
spark.read.format("delta").load(health_tracker + "silver").count()

10848

In [40]:
silver_health_tracker_data_df = spark.read \
    .format("delta") \
    .load(health_tracker + "silver")

In [41]:
from pyspark.sql.functions import col, mean, stddev, max
 
gold_health_tracker_data_df = silver_health_tracker_data_df.groupBy("device_id") \
          .agg(mean(col("heartrate")).alias("mean_heartrate"), 
               stddev(col("heartrate")).alias("std_heartrate"), 
               max(col("heartrate")).alias("max_heartrate"))


In [42]:
display(gold_health_tracker_data_df)

+---------+-----------------+------------------+--------------+
|device_id|   mean_heartrate|     std_heartrate| max_heartrate|
+---------+-----------------+------------------+--------------+
|        1|78.48811070421434| 32.41381560160755| 168.114687819|
|        3|82.21427876390008|30.386655126098105|175.0032148522|
|        4|81.52282375220065| 33.96101327923803| 199.092971234|
|        2|79.85712287206904|31.528706004038472|189.2113455089|
|        0|81.55085537215119|31.353747477803626|186.4790827731|
+---------+-----------------+------------------+--------------+



In [43]:
gold_health_tracker_data_df.write \
   .mode("overwrite") \
   .format("delta") \
   .partitionBy("device_id") \
   .save(health_tracker + "gold/heartrate_aggregation")


In [44]:
spark.sql("""
DROP TABLE IF EXISTS health_tracker_gold_heartrate_aggregation
""")

DataFrame[]

In [45]:
spark.sql(f"""
CREATE TABLE health_tracker_gold_heartrate_aggregation
USING DELTA
LOCATION "{health_tracker + "gold/heartrate_aggregation"}"
""")

DataFrame[]

In [46]:
raw_health_tracker_data_df = spark.read \
    .format("text") \
    .load(health_tracker + "raw/health_tracker_data_2020_3.json")

In [47]:
raw_health_tracker_data_df.write \
 .mode("append") \
 .format("delta") \
 .save(health_tracker + "bronze")

In [48]:
!rm ../data/health_tracker/raw/health_tracker_data_2020_3.json

In [49]:
bronze_health_tracker_data_df = spark.read \
    .format("delta") \
    .load(health_tracker + "bronze")

In [50]:
from pyspark.sql.functions import col, from_json

json_schema = "device_id int, heartrate double, name string, time float"
bronze_health_tracker_data_df = bronze_health_tracker_data_df \
    .withColumn("json", from_json(col("value"), json_schema))

In [51]:
silver_health_tracker_data_df = bronze_health_tracker_data_df \
    .select(
        col("json.device_id"),
        col("json.heartrate"),
        col("json.name"),
        col("json.time"),
    )

In [52]:
from pyspark.sql.functions import col, from_unixtime
 
silverDF = silver_health_tracker_data_df.withColumn("time", from_unixtime("time"))
silverDF = silverDF.withColumn("time", col("time").cast("timestamp"))
silverDF = silverDF.withColumn("dte", col("time").cast("date"))
silverDF = silverDF.withColumn("device_id", col("device_id").cast("integer"))


In [53]:
silver_health_tracker_data_df.write \
   .mode("append") \
   .format("delta") \
   .partitionBy("device_id") \
   .save(health_tracker + "silver")


In [54]:
spark.read.format("delta").load(health_tracker + "silver").count()

21696

In [55]:
silver_health_tracker_data_df = spark.read \
    .format("delta") \
    .load(health_tracker + "silver")

In [56]:
display(gold_health_tracker_data_df)

+---------+-----------------+-----------------+--------------+
|device_id|   mean_heartrate|    std_heartrate| max_heartrate|
+---------+-----------------+-----------------+--------------+
|        1|78.80687463997273|33.11572024824403|170.4458361857|
|        3|82.11862980332091|32.10612782852019|218.7730128322|
|        4|81.09156866022388|34.64492525161848| 199.092971234|
|        2|80.56347984456683|31.80555162296133|189.2113455089|
|        0|81.12243550089097| 31.3713313474477|186.4790827731|
+---------+-----------------+-----------------+--------------+

